# Summarize population density maps (`.tif`) by shapefiles (`.shp`)

### What does this do

This notebook allows you to:
1. Load a `.tif` population density file and a `.shp` shapefile

2. Sum the values of the pixels in the density map inside each given shape, resulting in a table of shape attributes (e.g. admin level names) and the respective population

3. Save the resulting table in a `.csv` file

### How to run
1. Install Python and Jupyter (recommended distribution: the latest `miniconda`)

2. Use `pip` to install the following packages if not already installed:
    - `rasterstats`
    
    - `rioxarray`

    - `pandas`

    - `geopandas`
    
    - `matplotlib`


3. Download your chosen population density map and shapefile and set the data paths here correctly

4. Run the `Setup` and `Functions` section and the relevant country and/or copy a country example cell and modify it to match your density map and shapefile.

# Setup

In [1]:
from pathlib import Path
import os

import rioxarray as rxr
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
from rasterstats import zonal_stats

## NECESSARY FUNCTION TO EXTEND `zonal_stats` TO WORK WITH XARRAYS
def zonal_stats_extended(gdf, xds, prefix=None, stats="sum", output_type="dataframe"):
    """
    Extends rasterstarts' zonal_stats to easily input xarrays and output a Pandas DataFrame.

    Default operation is "sum".

    """
    raster = xds.to_numpy()[0]
    affine = xds.rio.transform()
    nodata_value = xds.rio.nodata

    zs = zonal_stats(
        gdf,
        raster,
        affine=affine,
        nodata=nodata_value,
        stats=stats,
        prefix=prefix,
    )

    if output_type == "dataframe":
        return pd.DataFrame(zs)
    else:
        return zs

In [17]:
# Set paths to data folders
path = Path.cwd()
data_path = path / "Data"
output_path = path / "Output_Data"

In [5]:
current_directory = os.getcwd()
print(current_directory)

c:\Users\NOEL\Dropbox (IDinsight)\GiveWell\Program Reach\Phase 2\Code\pop-zonal-stats


# Full run

## Functions

In [22]:
def load_tif(path):
    """load raster population density data (.tif) with np.nan as nodata value"""

    return rxr.open_rasterio(path, masked=True)

In [23]:
def load_and_sum_tif(tif_file_path, gdf, prefix=None, rounding=True):
    """Load a pop density map from file and create zonal sums using shapes from the given gdf."""

    xds = load_tif(tif_file_path)

    zonal_stats_df = zonal_stats_extended(
        gdf=gdf,
        xds=xds,
        prefix=prefix,
        stats="sum"
    )

    if rounding:
        zonal_stats_df = zonal_stats_df.round().fillna(0).astype(int)

    return zonal_stats_df

In [25]:
def load_and_sum_multiple_tifs(
    gdf,
    tif_folder_path=None,
    tif_file_paths=None, 
    global_prefix=None, 
    file_prefix_list=None,
    rounding=True, 
    total_column=True
    ):
    """

    Primarily for WorldPop data since multiple files are provided, each for a different age/gender group.
    
    Load each density map file (either from a given folder or a list of file paths) and 
    get zonal sums of each based on a GeoDataFrame of shapes. Combine the results into 
    a single pandas dataframe and add a totals column if required.
    
    Parameters
    ----------
    gdf : geopandas.GeoDataFrame
        GeoDataFrame with shapes to summarize over.
    tif_folder_path : pathlib.Path, optional
        Path to folder with tif files to load. If provided, all tif files in the folder will be loaded and the 
        tif_file_paths parameter will be ignored.
    tif_file_paths : list, required if tif_folder_path is None
        List of paths to tif files.
    global_prefix : str, optional
        Prefix to add to all column names, by default None
    file_prefix_list : list, optional
        List of prefixes to add to each column name, by default None. If provided, must be the same length as the
        tif_file_paths list. If not provided, the file name will be used as the prefix.
    sum_rounding : bool, optional
        Whether to round sum values, by default True
    total_column : bool, optional
        Add a column with the total sum of all columns, by default True
        
    Returns
    -------
    pd.DataFrame
    
    """

    if tif_folder_path:
        tif_file_paths = list(tif_folder_path.glob("*.tif"))

    if not file_prefix_list:
        file_prefix_list = [f.stem for f in tif_file_paths]

    zonal_stats_df_list = []

    for tif_file_path, file_prefix in zip(tif_file_paths, file_prefix_list):
        prefix = global_prefix+file_prefix+"_"
        zonal_stats_df = load_and_sum_tif(tif_file_path, gdf, prefix, rounding)
        zonal_stats_df_list.append(zonal_stats_df)

    zonal_stats_df_all = pd.concat(zonal_stats_df_list, axis=1)

    if total_column:
        zonal_stats_df_all[global_prefix+"sum_total"] = zonal_stats_df_all.sum(axis=1)

    return zonal_stats_df_all

In [26]:
def combine_dataframes(
    gdf, 
    df_list, 
    shape_cols_to_keep, 
    save_csv, 
    output_csv_name="zonal_sums.csv"
    ):
    """Combine a list of dataframes with zonal sums with a GeoDataFrame of shapes and save result to file."""
    
    # combine all population dataframes
    pops_df = pd.concat(df_list, axis=1)

    # Combine with shape data
    gdf = gdf[shape_cols_to_keep+["geometry"]]
    combined_pops_gdf = pd.concat([gdf, pops_df], axis=1)

    # sort by area names
    combined_pops_gdf.sort_values(by=shape_cols_to_keep, inplace=True)

    ## save
    if save_csv:
        # save
        os.makedirs(output_path, exist_ok=True)
        combined_pops_gdf.drop(columns="geometry", axis=1).to_csv(output_path / output_csv_name, index=False)

    return combined_pops_gdf

For loading the three multiple data sources:

In [27]:
def create_combined_zonal_sums_for_all_sources(
    gdf,
    worldpop_cons_folder_path,
    worldpop_uncons_folder_path,
    facebook_file_path,
    landscan_file_path,
    shape_cols_to_keep,
    save_csv=True,
    output_csv_name="zonal_sums.csv",
    ):
    """Function to make zonal sums for each source and combine them into one table in one go."""
    
    # WorldPop Constrained
    worldpop_pops_df = load_and_sum_multiple_tifs(
        gdf=gdf,
        tif_folder_path=worldpop_cons_folder_path,
        global_prefix="worldpop_cons_u5_", 
        rounding=True, 
        total_column=True
        )
    
# WorldPop Unconstrained
    worldpop_pops_df = load_and_sum_multiple_tifs(
        gdf=gdf,
        tif_folder_path=worldpop_uncons_folder_path,
        global_prefix="worldpop_uncons_u5_", 
        rounding=True, 
        total_column=True
        )

    # Facebook
    facebook_pops_df = load_and_sum_tif(
        facebook_file_path,
        gdf,
        prefix="facebook_u5_",
        rounding=True
    )

    # LandScan. Note: TOTAL POPULATION, not under 5s.  
    landscan_pops_df = load_and_sum_tif(
        landscan_file_path,
        gdf,
        prefix="landscan_TOTAL_",
        rounding=True
    )

    df_list = [worldpop_pops_df, facebook_pops_df, landscan_pops_df]

    combined_pops_gdf = combine_dataframes(
        gdf, 
        df_list, 
        shape_cols_to_keep, 
        save_csv, 
        output_csv_name
        )

    return combined_pops_gdf

## WorldPop/FB/LandScan summaries
(Summarising multiple `.tif` density files and combining the results)

### 1. Nigeria

In [ ]:
nigeria_adm2_gdf = gpd.read_file(data_path / "Administrative Boundaries" / "nga_adm_osgof_20190417_nigeria" / "nga_admbnda_adm2_osgof_20190417.shp")

# plot shapes
# guinea_adm2_gdf.plot(color="white", edgecolor="gray", linewidth=0.5)
# guinea_adm2_gdf.head()

In [ ]:
worldpop_cons_folder_path=path/"Data"/ "WorldPop_Constrained_1km_UNadj/ Nigeria"
worldpop_uncons_folder_path=path/"Data"/ "WorldPop_Unconstrained_1km_UNadj"
facebook_file_path=path/"Data" / "Meta_HRSL" / "nga_children_under_five_2020_geotiff_Nigeria"
landscan_file_path=path/"Data"/ "Landscan"/ "landscan-global-2021.tif"

In [ ]:
nigeria_adm2_summary_gdf = create_combined_zonal_sums_for_all_sources(
    gdf=nigeria_adm2_gdf,
    worldpop_cons_folder_path=worldpop_cons_folder_path,
    worldpop_uncons_folder_path=worldpop_uncons_folder_path,
    facebook_file_path=facebook_file_path,
    landscan_file_path=landscan_file_path,
    shape_cols_to_keep=["admin1Name", "admin2Name"],
    save_csv=True,
    output_csv_name="nigeria_all_adm2.csv"
)

In [ ]:
# Plot map coloured by population summaries at the admin level of the shapefile.
# Note: for larger admin levels see next cell.
# guinea_pops_gdf.plot(column="landscan_total_u5", legend=True)

Note: We can also do this for higher admin levels, but this is more computationally expensive than simply doing a pivot table summary of the previously generated Admin level 2 data on the Admin level 1 column.

Directly doing the summary at higher levels does allow us to plot maps at the higher admin levels easier than going the pivot table route, however.

In [ ]:
# guinea_adm1_gdf = gpd.read_file(data_path / "boundaries" / "gin_admbnda_ocha_fis" / "gin_admbnda_adm1_ocha.shp")

# guinea_adm1_summary_gdf = create_combined_zonal_sums_for_all_sources(
#     gdf=guinea_adm1_gdf,
#     worldpop_folder_path=worldpop_folder_path,
#     facebook_file_path=facebook_file_path,
#     landscan_file_path=landscan_file_path,
#     shape_cols_to_keep=["admin1Name"],
#     save_csv=True,
#     output_csv_name="guinea_all_adm1.csv"
# )

In [35]:
# %%

# Basic Demo

*Case of Malaria Consortium in Nigeria using phase 2 estimates*

### 1. Load admin gdf


shapefile_path = data_path / "Administrative Boundaries" / "nga_adm_osgof_20190417_nigeria" / "nga_admbnda_adm2_osgof_20190417.shp"
admin_gdf = gpd.read_file(shapefile_path)

admin_gdf

admin_gdf.plot(color="white", edgecolor="gray", linewidth=0.5)

### 2. Load population density data

tif_file_path = path / "data" / "fb" / "population_phl_2018-10-01.tif"
pop_density_xarray = rxr.open_rasterio(tif_file_path, masked=True)

# pop_density_xarray.plot()

### 3. Sum population density per admin shape

In [ ]:
zonal_stats_df

### 4. Merge results with initial admin shapes

combined_pops_gdf = pd.concat([admin_gdf, zonal_stats_df], axis=1)

combined_pops_gdf

## Detailed general example: Philippines 
(Single `.tif` density file)

# load shape data
shapefile_path = data_path / "boundaries" / "phl_adminboundaries_candidate_adm3" / "phl_admbnda_adm3_psa_namria_20200529.shp"
philippines_adm3_gdf = gpd.read_file(shapefile_path)
print("Data columns in shapefile:\n", list(philippines_adm3_gdf.columns))

# plot shapes
philippines_adm3_gdf.plot(color="white", edgecolor="gray", linewidth=0.5)

# Based on the above list, set column names of shape attributes to keep in the final output
shape_cols_to_keep = ["ADM1_EN", "ADM2_EN", "ADM3_EN"]

tif_file_path = path / "data" / "fb" / "population_phl_2018-10-01.tif"
output_data_filename = "philippines_adm3_fb.csv"

# run the summing function (takes around 2 minutes to run)
philippines_adm3_populations = load_and_sum_tif(
    gdf=philippines_adm3_gdf,
    tif_file_path=tif_file_path,
    rounding=True,
    )

# combine with shape attribute data and save to file
philippines_adm3_populations_with_attributes = combine_dataframes(
    gdf=philippines_adm3_gdf,
    df_list=[philippines_adm3_populations],
    shape_cols_to_keep=shape_cols_to_keep,
    save_csv=True,
    output_csv_name=output_data_filename
    )

# Plot map coloured by population summaries at the admin level of the shapefile.
philippines_adm3_populations_with_attributes.plot(column="sum", legend=True)

zonal_stats_df = zonal_stats_extended(
    gdf=admin_gdf,
    xds=pop_density_xarray,
    prefix="pop_",
    stats="sum"
)

### Kenya

kenya_adm2_gdf = gpd.read_file(data_path / "boundaries" / "ken_adm_iebc_20191031_shp" / "ken_admbnda_adm2_iebc_20191031.shp")

worldpop_folder_path = path/"data"/"worldpop_constrained"/"kenya"
facebook_file_path = path/"data"/"fb"/"ken_children_under_five_2020.tif"
landscan_file_path = path/"data"/"landscan"/"landscan-global-2021-assets"/"landscan-global-2021.tif"

kenya_adm2_summary_gdf = create_combined_zonal_sums_for_all_sources(
    gdf=kenya_adm2_gdf,
    worldpop_folder_path=worldpop_folder_path,
    facebook_file_path=facebook_file_path,
    landscan_file_path=landscan_file_path,
    shape_cols_to_keep=["ADM1_EN", "ADM2_EN"],
    save_csv=True,
    output_csv_name="kenya_all_adm2.csv"
)

### Alternative Kenya data

In [ ]:
new_worldpop_folder_path = path/"data"/"worldpop_constrained"/"kenya" / "KEN_SAP_1km_2020"

new_kenya_adm2_summary_gdf = load_and_sum_multiple_tifs(
    gdf=kenya_adm2_gdf,
    tif_folder_path=new_worldpop_folder_path,
    global_prefix="worldpop_",
    rounding=True,
    total_column=True,
    )

combined_pops_gdf = combine_dataframes(
    gdf=kenya_adm2_gdf, 
    df_list=[new_kenya_adm2_summary_gdf], 
    shape_cols_to_keep=["ADM1_EN", "ADM2_EN"], 
    save_csv=True,
    output_csv_name="kenya_all_adm2_new.csv"
    )

### Burkina Faso

bfa_adm3_gdf = gpd.read_file(data_path / "boundaries" / "bfa_adm_igb_20200323_shp" / "bfa_admbnda_adm3_igb_20200323.shp")

worldpop_folder_path = path/"data"/"worldpop_constrained"/"burkina_faso"
facebook_file_path = path/"data"/"fb"/"bfa_children_under_five_2020.tif"
landscan_file_path = path/"data"/"landscan"/"landscan-global-2021-assets"/"landscan-global-2021.tif"

bfa_adm3_summary_gdf = create_combined_zonal_sums_for_all_sources(
    gdf=bfa_adm3_gdf,
    worldpop_folder_path=worldpop_folder_path,
    facebook_file_path=facebook_file_path,
    landscan_file_path=landscan_file_path,
    shape_cols_to_keep=["ADM1_FR", "ADM2_FR", "ADM3_FR"],
    save_csv=True,
    output_csv_name="burkina_faso_all_adm3.csv"
)

### Nigeria

nga_adm2_gdf = gpd.read_file(data_path / "boundaries" / "nga_adm_osgof_20190417" / "nga_admbnda_adm2_osgof_20190417.shp")

worldpop_folder_path = path/"data"/"worldpop_constrained"/"nigeria"
facebook_file_path = path/"data"/"fb"/"nga_children_under_five_2020.tif"
landscan_file_path = path/"data"/"landscan"/"landscan-global-2021-assets"/"landscan-global-2021.tif"

nga_adm2_summary_gdf = create_combined_zonal_sums_for_all_sources(
    gdf=nga_adm2_gdf,
    worldpop_folder_path=worldpop_folder_path,
    facebook_file_path=facebook_file_path,
    landscan_file_path=landscan_file_path,
    shape_cols_to_keep=["ADM1_EN", "ADM2_EN"],
    save_csv=True,
    output_csv_name="nigeria_all_adm2.csv"
)